In [3]:
import requests
import json
import googlemaps
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
from shapely.geometry import Point, Polygon
import shapely.geometry
import shapely.wkt
from shapely.geometry import LineString
from shapely.geometry import shape
import seaborn as sns

from matplotlib.colors import ListedColormap  
from mpl_toolkits.axes_grid1 import make_axes_locatable

#from pysal.lib import weights
import networkx as nx
#import momepy
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

import geopy.distance
from math import sin, cos, sqrt, atan2, radians
from numpy import arcsin as asin
from typing import Tuple
import mplleaflet
import contextily as cx
from datetime import datetime
import polyline

In [4]:
# enter your api key here
api_key = 'your API key '
gmaps = googlemaps.Client(key=api_key )

In [49]:
#departure at the current time, origin Kley Champs sur Marne, destinatino CIRED
now = datetime.now()
directions_result = gmaps.directions("Noisy - Champs", "CIRED IDF",
                                     mode="driving",
                                     departure_time=now)
#directions_result

# try to visualise the line

In [50]:
for i, leg in enumerate(directions_result[0]["legs"]):
    print("Stop:" + str(i),
        leg["start_address"], 
        "==> ",
        leg["end_address"], 
        "distance: ",  
        leg["distance"]["text"], 
        "traveling Time: ",
        leg["duration"]["text"]
    )

Stop:0 Noisy - Champs, Rue des Hauts Châteaux, 93160 Noisy-le-Grand, France ==>  94736 Cédex, 45bis Av. de la Belle Gabrielle, 94130 Nogent-sur-Marne, France distance:  12.1 km traveling Time:  20 mins


In [51]:
# get all the points between the OD
marker_points = []
waypoints = []
trunk = []
for leg in directions_result[0]["legs"]:
    leg_start_loc = leg["start_location"]
    marker_points.append(f'{leg_start_loc["lat"]},{leg_start_loc["lng"]}')
    for step in leg["steps"]:
        end_loc = step["end_location"]                     
        waypoints.append(f'{end_loc["lat"]},{end_loc["lng"]}')
        
        trunk.append(polyline.decode(step['polyline']['points']))
        
last_stop = directions_result[0]["legs"][-1]["end_location"]
marker_points.append(f'{last_stop["lat"]},{last_stop["lng"]}')

In [52]:
#change the format of the output, adapt to the gmaps format for plotting
itinerary = []
for curve in trunk:
    for point in curve:
        itinerary.append(f'{point[0]},{point[1]}')
#itinerary   

In [53]:
markers = [ "color:blue|size:mid|label:" + chr(65+i) + "|" 
           + r for i, r in enumerate(marker_points)]
result_map = gmaps.static_map(
                 center = waypoints[0],
                 scale=2, 
                 zoom=11,
                 size=[640, 640], 
                 format="jpg", 
                 maptype="roadmap",
                 markers=markers,
                 path="color:blue|weight:3|" + "|".join(itinerary ))

In [54]:
with open("car_route_map.jpg", "wb") as img:
    for chunk in result_map:
        img.write(chunk)